In [ ]:
import os
from pathlib import Path
from PIL import Image

from google.colab import drive
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
from sklearn.metrics import accuracy_score


In [ ]:
# подключаем GDrive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gpiosenka/100-bird-species

 99% 1.47G/1.49G [00:07<00:00, 194MB/s]
100% 1.49G/1.49G [00:07<00:00, 215MB/s]


In [ ]:
!unzip 100-bird-species.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/VERMILION FLYCATHER/102.jpg  
  inflating: train/VERMILION FLYCATHER/103.jpg  
  inflating: train/VERMILION FLYCATHER/104.jpg  
  inflating: train/VERMILION FLYCATHER/105.jpg  
  inflating: train/VERMILION FLYCATHER/106.jpg  
  inflating: train/VERMILION FLYCATHER/107.jpg  
  inflating: train/VERMILION FLYCATHER/108.jpg  
  inflating: train/VERMILION FLYCATHER/109.jpg  
  inflating: train/VERMILION FLYCATHER/110.jpg  
  inflating: train/VERMILION FLYCATHER/111.jpg  
  inflating: train/VERMILION FLYCATHER/112.jpg  
  inflating: train/VERMILION FLYCATHER/113.jpg  
  inflating: train/VERMILION FLYCATHER/114.jpg  
  inflating: train/VERMILION FLYCATHER/115.jpg  
  inflating: train/VERMILION FLYCATHER/116.jpg  
  inflating: train/VERMILION FLYCATHER/117.jpg  
  inflating: train/VERMILION FLYCATHER/118.jpg  
  inflating: train/VERMILION FLYCATHER/119.jpg  
  inflating: train/VERMILION FLYCATHER/120.jpg  
 

In [ ]:
data_path = Path(".")

In [ ]:
train_data_path = data_path / "train"
valid_data_path = data_path / "valid"
test_data_path = data_path / "test"

In [ ]:
class_to_idx = {}
for i, directory in enumerate(os.listdir(train_data_path)):
    # YOUR CODE
    class_to_idx[directory] = i

In [ ]:

class ImageDataset(Dataset):
    
    def __init__(self, file_path, transform, class_to_idx):
        self.files = list(file_path.rglob("*.jpg"))# подставьте маску для файлов
        self.transform = transform
        self.class_to_idx = class_to_idx
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        img = Image.open(file)
        tensor = self.transform(img)
        label = file.parent.name 
        label_id = self.class_to_idx.get(label,-1) # если label в self.class_to_idx то возращаем его id если нет то -1
        return tensor, label_id

In [ ]:
# используйте разные трансформации train чтобы для того чтобы данные были более различны можно почитать про AutoAgument https://pytorch.org/vision/stable/transforms.html#automatic-augmentation-transforms
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])


valid_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),  

    transforms.CenterCrop(224),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_dataset = ImageDataset(train_data_path, train_transform, class_to_idx)
valid_dataset = ImageDataset(valid_data_path, valid_transform, class_to_idx)
test_dataset = ImageDataset(test_data_path, valid_transform, class_to_idx)

In [ ]:
# Определим параметры обучения

BATCH_SIZE = 100
EPOCH = 10
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 1e-5

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [ ]:
num_classes = 400
modell_pretrained = models.alexnet(pretrained=True) 
modell_pretrained.fc = nn.Linear(modell_pretrained.fc.in_features, num_classes)
modell_pretrained = modell_pretrained.to("cuda") 

In [ ]:
optimizer = optim.Adam(net_pretrained.parameters(), weight_decay = WEIGHT_DECAY, lr = LEARNING_RATE)

In [ ]:
criteria = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(EPOCH):
    modell_pretrained.train()
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        x = x.to("cuda")
        y = y.to("cuda")
        output = modell_pretrained.forward(x.float())
        loss = criteria(output, y)
        loss.backward() 
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch + 1} / 10, iteration {i + 1} / {len(train_loader)}, loss: {loss.item():.3f}")
        
    y_true = []
    y_pred = []
    
    model_pretrained.eval() # перевести сеть в режим предсказаний
    for i, (x, y) in enumerate(valid_loader):
            x = x.to("cuda") # переводим x на cuda
            y = y.numpy().flatten() # переводим y в numpy массив
            output = modell_pretrained.forward(x.float()) 
            output = output.cpu().numpy().flatten()
            y_true.extend(y)
            y_pred.extend(output.argmax(1))
            
    score = accuracy_score(y_true, y_pred)
    print(f"Epoch {epoch + 1} / 10, accuracy: {score:.3f}")

Epoch 1 / 10, iteration 100 / 266, loss: 1.172
Epoch 1 / 10, iteration 200 / 266, loss: 0.682
Epoch 1 / 10, accuracy: 0.794
Epoch 2 / 10, iteration 100 / 266, loss: 0.292
Epoch 2 / 10, iteration 200 / 266, loss: 0.356
Epoch 2 / 10, accuracy: 0.898
Epoch 3 / 10, iteration 100 / 266, loss: 0.146
Epoch 3 / 10, iteration 200 / 266, loss: 0.172
Epoch 3 / 10, accuracy: 0.887
Epoch 4 / 10, iteration 100 / 266, loss: 0.055
Epoch 4 / 10, iteration 200 / 266, loss: 0.166
Epoch 4 / 10, accuracy: 0.896


In [ ]:
for i, (x, y) in enumerate(test_loader):
    with torch.no_grad():
        x = x.to("cuda")
        y = y.flatten()
        output = modell_pretrained.forward(x.float())
        output = output.argmax(1).cpu().numpy()
        y_true.extend(y)
        y_pred.extend(output)
score = accuracy_score(y_true, y_pred)
print(f"Test accuracy: {score:.3f}")

Test accuracy: 0.896
